In [14]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/amazon-ml/sample_test.csv
/kaggle/input/amazon-ml/sample_test_out_fail.csv
/kaggle/input/amazon-ml/sample_test_out.csv
/kaggle/input/amazon-ml/test_out_third_third.csv
/kaggle/input/amazon-ml/test_out_second_third.csv
/kaggle/input/amazon-ml/train.csv
/kaggle/input/amazon-ml/test_out.csv
/kaggle/input/amazon-ml/test.csv
/kaggle/input/amazon-ml-images/filtered_image_data.csv
/kaggle/input/amazon-ml-images/downloaded_images/downloaded_images/81OUaAc42CS.jpg
/kaggle/input/amazon-ml-images/downloaded_images/downloaded_images/71D81Xct+vL.jpg
/kaggle/input/amazon-ml-images/downloaded_images/downloaded_images/81iRmDoi-mL.jpg
/kaggle/input/amazon-ml-images/downloaded_images/downloaded_images/61lyjKjb5xL.jpg
/kaggle/input/amazon-ml-images/downloaded_images/downloaded_images/51Xfq-wnHrL.jpg
/kaggle/input/amazon-ml-images/downloaded_images/downloaded_images/61TZ6ABywkL.jpg
/kaggle/input/amazon-ml-images/downloaded_images/downloaded_images/511Wc62KoiL.jpg
/kaggle/input/amazon-ml-imag

In [15]:
# import warnings
# import requests
# from io import BytesIO

# import pandas as pd
# import numpy as np

# from sklearn.preprocessing import LabelEncoder
# from sklearn.model_selection import train_test_split

# from PIL import Image
# import albumentations as A
# from albumentations.pytorch import ToTensorV2
# from tqdm import tqdm

# import easyocr

# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# from torch.utils.data import Dataset, DataLoader
# from torch.cuda.amp import autocast, GradScaler
# from torchvision import models

# from transformers import (
#     AutoTokenizer, 
#     AutoModel, 
#     AdamW, 
#     get_linear_schedule_with_warmup, 
#     BlipProcessor, 
#     BlipForImageTextRetrieval
# )

# print("Libraries Imported!")

# warnings.filterwarnings('ignore')
# torch.backends.cudnn.benchmark = True

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print(f"Using device: {device}")

# # Hyperparameters
# BATCH_SIZE = 32
# EPOCHS = 20
# LEARNING_RATE = 1e-5
# MAX_LEN = 128
# IMAGE_SIZE = 224

# entity_unit_map = {
#     "width": {"centimetre", "foot", "millimetre", "metre", "inch", "yard"},
#     "depth": {"centimetre", "foot", "millimetre", "metre", "inch", "yard"},
#     "height": {"centimetre", "foot", "millimetre", "metre", "inch", "yard"},
#     "item_weight": {"milligram", "kilogram", "microgram", "gram", "ounce", "ton", "pound"},
#     "maximum_weight_recommendation": {"milligram", "kilogram", "microgram", "gram", "ounce", "ton", "pound"},
#     "voltage": {"millivolt", "kilovolt", "volt"},
#     "wattage": {"kilowatt", "watt"},
#     "item_volume": {"cubic foot", "microlitre", "cup", "fluid ounce", "centilitre", "imperial gallon", "pint", "decilitre", "litre", "millilitre", "quart", "cubic inch", "gallon"}
# }

# all_units = set.union(*entity_unit_map.values())
# all_units.add("unknown")

# unit_encoder = LabelEncoder()
# unit_encoder.fit(list(all_units))

# label_encoder = LabelEncoder()

# class ProductImageDataset(Dataset):
#     def __init__(self, df, tokenizer, max_len, is_test=False):
#         self.df = df
#         self.tokenizer = tokenizer
#         self.max_len = max_len
#         self.is_test = is_test
#         self.transform = A.Compose([
#             A.Resize(IMAGE_SIZE, IMAGE_SIZE),
#             A.HorizontalFlip(p=0.5),
#             A.VerticalFlip(p=0.5),
#             A.RandomBrightnessContrast(p=0.5),
#             A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
#             ToTensorV2()
#         ])
#         if not is_test:
#             self.df['encoded_entity_name'] = label_encoder.transform(self.df['entity_name'])

#     def __len__(self):
#         return len(self.df)

#     @staticmethod
#     def download_image(url):
#         try:
#             response = requests.get(url, timeout=5)
#             img = Image.open(BytesIO(response.content)).convert('RGB')
#             return np.array(img)
#         except:
#             return np.zeros((IMAGE_SIZE, IMAGE_SIZE, 3), dtype=np.uint8)

#     def __getitem__(self, idx):
#         image_url = self.df.iloc[idx]['image_link']
#         entity_name = self.df.iloc[idx]['entity_name']
        
#         img = self.download_image(image_url)
#         img = self.transform(image=img)['image']

#         combined_text = f"{entity_name}"

#         encoding = self.tokenizer.encode_plus(
#             combined_text,
#             add_special_tokens=True,
#             max_length=self.max_len,
#             padding='max_length',
#             return_attention_mask=True,
#             truncation=True,
#             return_tensors='pt'
#         )

#         item = {
#             'image': img,
#             'input_ids': encoding['input_ids'].flatten(),
#             'attention_mask': encoding['attention_mask'].flatten(),
#             'entity_name': entity_name
#         }

#         if not self.is_test:
#             item['entity_label'] = torch.tensor(self.df.iloc[idx]['encoded_entity_name'], dtype=torch.long)
            
#             entity_value = self.df.iloc[idx]['entity_value']
#             value, unit = self.parse_entity_value(entity_value)
#             item['entity_value'] = torch.tensor(value, dtype=torch.float)
            
#             if unit == "":
#                 unit = "unknown"
#             item['entity_unit'] = torch.tensor(unit_encoder.transform([unit])[0], dtype=torch.long)

#         return item

#     @staticmethod
#     def parse_entity_value(entity_value):
#         parts = entity_value.replace('[', '').replace(']', '').split()
#         values = [float(part) for part in parts if part.replace('.', '').isdigit()]
#         value = sum(values) / len(values) if values else 0.0
#         unit = ' '.join(parts[len(values):]).strip()
#         return value, unit

# class VLMOCRNLPEntityExtractionModel(nn.Module):
#     def __init__(self, num_labels, num_units):
#         super(VLMOCRNLPEntityExtractionModel, self).__init__()
#         self.num_labels = num_labels
#         self.num_units = num_units
        
#         # BLIP Component
#         self.blip_model = BlipForImageTextRetrieval.from_pretrained("Salesforce/blip-itm-base-coco")
#         self.blip_processor = BlipProcessor.from_pretrained("Salesforce/blip-itm-base-coco")
        
#         # Freeze BLIP parameters
#         for param in self.blip_model.parameters():
#             param.requires_grad = False
        
#         # OCR Component
#         self.ocr_reader = easyocr.Reader(['en'])
        
#         # NLP Component (using DeBERTa as an example)
#         self.nlp_tokenizer = AutoTokenizer.from_pretrained('microsoft/deberta-v3-base')
#         self.nlp_model = AutoModel.from_pretrained('microsoft/deberta-v3-base')
        
#         # Fusion and output layers
#         self.fusion_layer = nn.Linear(self.blip_model.config.hidden_size + self.nlp_model.config.hidden_size, 1024)
#         self.classifier = nn.Linear(1024, num_labels)
#         self.regressor = nn.Linear(1024, 1)
#         self.unit_classifier = nn.Linear(1024, num_units)
        
#         self.dropout = nn.Dropout(0.3)

#         # Weight initialization
#         self._init_weights()

#     def _init_weights(self):
#         for module in [self.fusion_layer, self.classifier, self.regressor, self.unit_classifier]:
#             if isinstance(module, nn.Linear):
#                 nn.init.xavier_uniform_(module.weight)
#                 if module.bias is not None:
#                     nn.init.constant_(module.bias, 0)

#     def forward(self, images, entity_names):
#         # Process images with BLIP
#         blip_inputs = self.blip_processor(images=images, return_tensors="pt", padding=True).to(images.device)
#         with torch.no_grad():
#             blip_outputs = self.blip_model.vision_model(**blip_inputs)
#         blip_features = blip_outputs.pooler_output
        
#         # Perform OCR on images
#         ocr_texts = []
#         for img in images:
#             img_np = img.permute(1, 2, 0).cpu().numpy()
#             img_pil = Image.fromarray((img_np * 255).astype('uint8'))
#             result = self.ocr_reader.readtext(img_pil)
#             ocr_text = ' '.join([text for _, text, _ in result])
#             ocr_texts.append(ocr_text)
        
#         # Combine OCR text with entity names
#         combined_texts = [f"{entity_name} {ocr_text}" for entity_name, ocr_text in zip(entity_names, ocr_texts)]
        
#         # Process combined text with NLP model
#         nlp_inputs = self.nlp_tokenizer(combined_texts, return_tensors="pt", padding=True, truncation=True).to(images.device)
#         nlp_outputs = self.nlp_model(**nlp_inputs).last_hidden_state[:, 0, :]  # Use [CLS] token representation
        
#         # Fuse VLM and NLP outputs
#         fused_features = torch.cat((blip_features, nlp_outputs), dim=1)
#         fused_features = self.dropout(fused_features)
#         fused_features = F.relu(self.fusion_layer(fused_features))
#         fused_features = self.dropout(fused_features)
        
#         # Generate outputs
#         logits = self.classifier(fused_features)
#         value = self.regressor(fused_features).squeeze(-1)
#         unit_logits = self.unit_classifier(fused_features)
        
#         return logits, value, unit_logits

# def train_model(model, train_loader, val_loader, optimizer, scheduler, classification_criterion, regression_criterion, unit_criterion, epochs, device):
#     scaler = torch.cuda.amp.GradScaler()
#     best_val_loss = float('inf')
    
#     for epoch in range(epochs):
#         model.train()
#         total_loss = 0
#         total_class_loss = 0
#         total_reg_loss = 0
#         total_unit_loss = 0
        
#         for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
#             optimizer.zero_grad(set_to_none=True)

#             images = batch['image'].to(device)
#             entity_names = batch['entity_name']
#             labels = batch['entity_label'].to(device)
#             values = batch['entity_value'].to(device)
#             units = batch['entity_unit'].to(device)

#             with torch.cuda.amp.autocast():
#                 logits, predicted_values, unit_logits = model(images, entity_names)
#                 classification_loss = classification_criterion(logits, labels)
#                 regression_loss = regression_criterion(predicted_values, values)
#                 unit_loss = unit_criterion(unit_logits, units)
#                 loss = classification_loss + regression_loss + unit_loss

#             scaler.scale(loss).backward()
#             scaler.step(optimizer)
#             scaler.update()

#             total_loss += loss.item()
#             total_class_loss += classification_loss.item()
#             total_reg_loss += regression_loss.item()
#             total_unit_loss += unit_loss.item()

#         scheduler.step(total_loss)

#         avg_train_loss = total_loss / len(train_loader)
#         avg_class_loss = total_class_loss / len(train_loader)
#         avg_reg_loss = total_reg_loss / len(train_loader)
#         avg_unit_loss = total_unit_loss / len(train_loader)
        
#         val_loss, val_class_loss, val_reg_loss, val_unit_loss = validate_model(model, val_loader, classification_criterion, regression_criterion, unit_criterion, device)
        
#         print(f'Epoch {epoch + 1}/{epochs}')
#         print(f'Train Loss: {avg_train_loss:.4f} (Class: {avg_class_loss:.4f}, Reg: {avg_reg_loss:.4f}, Unit: {avg_unit_loss:.4f})')
#         print(f'Val Loss: {val_loss:.4f} (Class: {val_class_loss:.4f}, Reg: {val_reg_loss:.4f}, Unit: {val_unit_loss:.4f})')
        
#         if val_loss < best_val_loss:
#             best_val_loss = val_loss
#             torch.save(model.state_dict(), 'best_model.pth')
#             print("Saved best model.")

#     return model

# def validate_model(model, val_loader, classification_criterion, regression_criterion, unit_criterion):
#     model.eval()
#     val_loss = 0
#     val_class_loss = 0
#     val_reg_loss = 0
#     val_unit_loss = 0
#     with torch.no_grad():
#         for batch in val_loader:
#             input_ids = batch['input_ids'].to(device)
#             attention_mask = batch['attention_mask'].to(device)
#             images = batch['image'].to(device)
#             labels = batch['entity_label'].to(device)
#             values = batch['entity_value'].to(device)
#             units = batch['entity_unit'].to(device)

#             with autocast():
#                 logits, predicted_values, unit_logits = model(input_ids=input_ids, attention_mask=attention_mask, images=images)
#                 classification_loss = classification_criterion(logits, labels)
#                 regression_loss = regression_criterion(predicted_values, values)
#                 unit_loss = unit_criterion(unit_logits, units)
#                 loss = classification_loss + regression_loss + unit_loss
#             val_loss += loss.item()
#             val_class_loss += classification_loss.item()
#             val_reg_loss += regression_loss.item()
#             val_unit_loss += unit_loss.item()

#     return (val_loss / len(val_loader), 
#             val_class_loss / len(val_loader), 
#             val_reg_loss / len(val_loader), 
#             val_unit_loss / len(val_loader))

# def inference(model, test_loader, tokenizer):
#     model.eval()
#     predictions = []
    
#     with torch.no_grad():
#         for batch in tqdm(test_loader, desc="Inference"):
#             input_ids = batch['input_ids'].to(device)
#             attention_mask = batch['attention_mask'].to(device)
#             images = batch['image'].to(device)
#             entity_names = batch['entity_name']

#             with autocast():
#                 logits, predicted_values, unit_logits = model(input_ids=input_ids, attention_mask=attention_mask, images=images)
            
#             pred_entity_names = torch.argmax(logits, dim=1)
#             pred_entity_names = label_encoder.inverse_transform(pred_entity_names.cpu().numpy())
            
#             pred_units = torch.argmax(unit_logits, dim=1)
#             pred_units = unit_encoder.inverse_transform(pred_units.cpu().numpy())
            
#             for idx, (entity_name, value, unit) in enumerate(zip(entity_names, predicted_values.cpu().numpy(), pred_units)):
#                 formatted_value = format_value(value, unit, entity_name)
#                 predictions.append({
#                     'index': idx,
#                     'prediction': formatted_value
#                 })

#     return pd.DataFrame(predictions)

# def format_value(value, unit, entity_name):
#     if entity_name not in entity_unit_map or unit not in entity_unit_map[entity_name]:
#         return ""
#     return f"{value:.2f} {unit}".strip()

# def main():
#     device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#     print(f"Using device: {device}")

#     nlp_tokenizer = AutoTokenizer.from_pretrained('microsoft/deberta-v3-base')
    
#     train_df = pd.read_csv('/kaggle/input/train.csv')
#     test_df = pd.read_csv('/kaggle/input/test.csv')
    
#     train_df = train_df.head(5000)
    
#     # Filter rows based on allowed units
#     train_df['unit'] = train_df['entity_value'].str.split().str[-1]
#     train_df = train_df[train_df['unit'].isin(all_units)]
    
#     train_df, val_df = train_test_split(train_df, test_size=0.1, random_state=42)
    
#     label_encoder.fit(train_df['entity_name'])

#     print("Preparing datasets...")
#     train_dataset = ProductImageDataset(train_df, nlp_tokenizer, MAX_LEN)
#     val_dataset = ProductImageDataset(val_df, nlp_tokenizer, MAX_LEN)
#     test_dataset = ProductImageDataset(test_df, nlp_tokenizer, MAX_LEN, is_test=True)

#     print("Creating data loaders...")
#     train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True)
#     val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True)
#     test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True)

#     num_labels = len(label_encoder.classes_)
#     num_units = len(unit_encoder.classes_)
#     model = VLMOCRNLPEntityExtractionModel(num_labels=num_labels, num_units=num_units).to(device)

#     optimizer = AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=0.01)
#     scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)
    
#     classification_criterion = nn.CrossEntropyLoss().to(device)
#     regression_criterion = nn.MSELoss().to(device)
#     unit_criterion = nn.CrossEntropyLoss().to(device)

#     print("Starting training...")
#     model = train_model(model, train_loader, val_loader, optimizer, scheduler, classification_criterion, regression_criterion, unit_criterion, EPOCHS, device)

#     print("Loading best model for inference...")
#     model.load_state_dict(torch.load('best_model.pth'))

#     print("Running inference on test set...")
#     predictions_df = inference(model, test_loader, nlp_tokenizer, device)
#     predictions_df.to_csv('submission.csv', index=False)
#     print("Predictions saved to submission.csv")

In [16]:
# main()

# Submission

In [17]:
# import os
# import pandas as pd
# import torch
# from tqdm import tqdm
# from torch.cuda.amp import autocast
# import random

# # Assuming these constants and objects are available
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# def format_value(value, unit, entity_name):
#     """Format the predicted value based on unit and entity_name."""
#     if entity_name not in entity_unit_map or unit not in entity_unit_map[entity_name]:
#         return ""
#     return f"{value:.2f} {unit}".strip()

# def predictor(image_link, category_id, entity_name):
#     '''
#     A simple random predictor. Replace this with actual model predictions.
#     '''
#     # For testing purposes, we'll generate random predictions.
#     # Replace this with actual inference logic.
#     if random.random() > 0.5:
#         return f"{random.uniform(1, 100):.2f} inch"
#     return ""

# def inference(model, test_loader, tokenizer):
#     """This function handles the model inference on test data."""
#     model.eval()
#     predictions = []

#     with torch.no_grad():
#         for batch in tqdm(test_loader, desc="Inference"):
#             input_ids = batch['input_ids'].to(device)
#             attention_mask = batch['attention_mask'].to(device)
#             images = batch['image'].to(device)
#             entity_names = batch['entity_name']

#             with autocast():
#                 logits, predicted_values, unit_logits = model(input_ids=input_ids, attention_mask=attention_mask, images=images)

#             # Convert the logits to meaningful values (labels and units)
#             pred_entity_names = torch.argmax(logits, dim=1)
#             pred_entity_names = label_encoder.inverse_transform(pred_entity_names.cpu().numpy())

#             pred_units = torch.argmax(unit_logits, dim=1)
#             pred_units = unit_encoder.inverse_transform(pred_units.cpu().numpy())

#             for idx, (entity_name, value, unit) in enumerate(zip(entity_names, predicted_values.cpu().numpy(), pred_units)):
#                 formatted_value = format_value(value, unit, entity_name)
#                 predictions.append({
#                     'index': idx,
#                     'prediction': formatted_value
#                 })

#     return pd.DataFrame(predictions)

# def run_test_prediction():
#     """Run the prediction and save to submission.csv"""
#     DATASET_FOLDER = '/kaggle/input/amazon-ml'  # Path to your dataset folder
#     WORKING_DIR = '/kaggle/working/'
    
#     # Step 1: Load test data
#     test = pd.read_csv(os.path.join(DATASET_FOLDER, 'test.csv'))
    
#     # Step 2: Apply predictor function on test data
#     test['prediction'] = test.apply(
#         lambda row: predictor(row['image_link'], row['group_id'], row['entity_name']), axis=1)

#     # Step 3: Ensure predictions are formatted correctly
#     test['prediction'] = test['prediction'].apply(
#         lambda x: x if isinstance(x, str) and x else "")

#     # Step 4: Save the predictions to submission.csv
#     output_filename = os.path.join(WORKING_DIR, 'submission.csv')
#     test[['index', 'prediction']].to_csv(output_filename, index=False)

#     print(f"Submission saved to {output_filename}")


# run_test_prediction()

In [18]:
!pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-o14njoc1
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-o14njoc1
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done


In [ ]:
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
from sklearn.model_selection import train_test_split
import easyocr
import clip
import logging

In [ ]:
# Set up logging
logging.basicConfig(level=logging.INFO)

# Constants
ROOT_DIR = "/kaggle/input/amazon-ml-images/downloaded_images/downloaded_images"
CSV_PATH = "/kaggle/input/amazon-ml-images/filtered_image_data.csv"

# Entity unit map (as provided in your code)
entity_unit_map = {
    'width': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'depth': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'height': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'item_weight': {'gram', 'kilogram', 'microgram', 'milligram', 'ounce', 'pound', 'ton'},
    'maximum_weight_recommendation': {'gram', 'kilogram', 'microgram', 'milligram', 'ounce', 'pound', 'ton'},
    'voltage': {'kilovolt', 'millivolt', 'volt'},
    'wattage': {'kilowatt', 'watt'},
    'item_volume': {'centilitre', 'cubic foot', 'cubic inch', 'cup', 'decilitre', 'fluid ounce', 'gallon', 'imperial gallon', 'litre', 'microlitre', 'millilitre', 'pint', 'quart'}
}

allowed_units = {unit for entity in entity_unit_map for unit in entity_unit_map[entity]}

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
logging.info(f"Using device: {device}")

# Initialize EasyOCR reader
reader = easyocr.Reader(['en'])

In [ ]:
# Load and preprocess data
try:
    df = pd.read_csv(CSV_PATH)
    train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)
    logging.info(f"Loaded {len(df)} samples, split into {len(train_df)} train and {len(val_df)} validation samples")
except Exception as e:
    logging.error(f"Error loading CSV file: {e}")
    raise

In [ ]:
class EntityExtractionDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        image_path = os.path.join(ROOT_DIR, row['filename'])
        group_id = row['group_id']
        entity_name = row['entity_name']
        entity_value = row['entity_value']

        try:
            image = Image.open(image_path).convert('RGB')
            if self.transform:
                image = self.transform(image)
        except Exception as e:
            logging.error(f"Error loading image {image_path}: {e}")
            # Return a placeholder image or skip this sample
            image = torch.zeros((3, 224, 224))

        return image, group_id, entity_name, entity_value

# Define transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Create datasets and dataloaders
train_dataset = EntityExtractionDataset(train_df, transform=transform)
val_dataset = EntityExtractionDataset(val_df, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)

In [ ]:
class OCRModule(nn.Module):
    def __init__(self):
        super(OCRModule, self).__init__()
        self.reader = reader

    def forward(self, images):
        batch_text = []
        for img in images:
            img_np = (img.permute(1, 2, 0).cpu().numpy() * 255).astype('uint8')
            try:
                result = self.reader.readtext(img_np)
                text = ' '.join([item[1] for item in result])
            except Exception as e:
                logging.error(f"OCR error: {e}")
                text = ""
            batch_text.append(text)
        return batch_text

class EntityExtractionModel(nn.Module):
    def __init__(self, num_classes):
        super(EntityExtractionModel, self).__init__()
        self.clip_model, _ = clip.load("ViT-B/32", device=device)
        self.ocr_module = OCRModule()
        
        # Freeze CLIP parameters
        for param in self.clip_model.parameters():
            param.requires_grad = False
        
        # Define custom layers with the same dtype as CLIP
        self.fc1 = nn.Linear(512 * 2, 256).to(self.clip_model.dtype)
        self.fc2 = nn.Linear(256, num_classes).to(self.clip_model.dtype)
        self.relu = nn.ReLU()

    def forward(self, image, group_id, entity_name):
        image_features = self.clip_model.encode_image(image)
        
        # Perform OCR
        ocr_text = self.ocr_module(image)
        
        # Combine OCR text with group_id and entity_name
        combined_text = [f"{ot} Group ID: {gid} Entity: {en}" for ot, gid, en in zip(ocr_text, group_id, entity_name)]
        
        text_features = self.clip_model.encode_text(clip.tokenize(combined_text).to(device))
        
        combined_features = torch.cat((image_features, text_features), dim=1)
        x = self.relu(self.fc1(combined_features))
        return self.fc2(x)

In [ ]:
# Create model, loss function, and optimizer
model = EntityExtractionModel(num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Convert model to float16 if using GPU
if device.type == 'cuda':
    model.half()
    criterion.half()

# Create hierarchical classification structure
entity_to_units = {entity: list(units) for entity, units in entity_unit_map.items()}
all_units = [unit for units in entity_to_units.values() for unit in units]
unit_to_idx = {unit: idx for idx, unit in enumerate(all_units)}
num_classes = len(all_units)

In [21]:
# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    for i, (images, group_ids, entity_names, entity_values) in enumerate(train_loader):
        # In the training loop
        images = images.to(device)
        
        optimizer.zero_grad()
        outputs = model(images, group_ids, entity_names)
        
        # Convert entity_values to class indices
        target_indices = torch.tensor([unit_to_idx[value.split()[1]] for value in entity_values])
        target_indices = target_indices.to(device)
        
        loss = criterion(outputs, target_indices)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        
        if i % 100 == 0:
            logging.info(f"Epoch [{epoch+1}/{num_epochs}], Step [{i}/{len(train_loader)}], Loss: {loss.item():.4f}")

    # Validation
    model.eval()
    val_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, group_ids, entity_names, entity_values in val_loader:
            images = images.to(device)
            
            outputs = model(images, group_ids, entity_names)
            
            target_indices = torch.tensor([unit_to_idx[value.split()[1]] for value in entity_values])
            target_indices = target_indices.to(device)
            
            loss = criterion(outputs, target_indices)
            val_loss += loss.item()
            
            _, predicted = outputs.max(1)
            total += target_indices.size(0)
            correct += predicted.eq(target_indices).sum().item()

    avg_train_loss = train_loss / len(train_loader)
    avg_val_loss = val_loss / len(val_loader)
    accuracy = correct / total

    logging.info(f"Epoch [{epoch+1}/{num_epochs}], "
                 f"Train Loss: {avg_train_loss:.4f}, "
                 f"Val Loss: {avg_val_loss:.4f}, "
                 f"Val Accuracy: {accuracy:.4f}")

    # Save the model after each epoch
    torch.save(model.state_dict(), f"entity_extraction_model_epoch_{epoch+1}.pth")

logging.info("Training completed and model saved.")

/opt/conda/lib/python3.10/site-packages/easyocr/detection.py:85: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(copyStateDict(torch.load(trained_model, ma

RuntimeError: expected scalar type Float but found Half